# Processing monocular and binocular data

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,"..")
import pypillometry as pp
import numpy as np

Data in `pypillometry` can contain different `variables` from different `eyes`. The variables and eyes supported when importing raw data are

- `left_x`, `right_x` (x-coordinate in screen coordinates from the eyetracker)
- `left_y`, `right_y` (y-coordinate in screen coordinates from the eyetracker)
- `left_pupil`, `right_pupil` (pupil size from left and right eye)

Depending on which class is chosen (`PupilData`, `GazeData` or `EyeData`), some of these variables are required:

- `PupilData`: requires at least one of `left_pupil`, `right_pupil` (or both)
- `GazeData`: requires at least one of `(left_x, left_y)` and/or `(right_x, right_y)`
- `EyeData`: requires `x`,`y` and `pupil` from at least one eye

For example, let's simulate some basic data:

In [2]:
left_x = np.random.randn(1000)
right_x = np.random.randn(1000)
left_y = np.random.randn(1000)
right_y = np.random.randn(1000)
left_pupil = np.random.randn(1000)
right_pupil = np.random.randn(1000)
time = np.arange(1000)

# these are all ok
dpupil = pp.PupilData(left_pupil=left_pupil, right_pupil=right_pupil, time=time)
dgaze = pp.GazeData(left_x=left_x, left_y=left_y, right_x=right_x, right_y=right_y, time=time)
deye = pp.EyeData(left_x=left_x, left_y=left_y, left_pupil=left_pupil, time=time)

# these are not ok
#pp.PupilData(left_x=left_x, time=time)
#pp.GazeData(left_x=left_x, time=time)
pp.EyeData(left_x=left_x, time=time)

ValueError: At least one of the eye-traces must be provided (both x and y)

Once the data is loaded, we can check which variables and eyes are available using the `.eyes` and `.variables` attribute:

In [18]:
deye.eyes, deye.variables

(['left'], ['y', 'x', 'pupil'])

Simply printing an object will also show what data sources are available and give a glimpse into the data structure:

In [19]:
deye

EyeData(mohavedi, 55.6KiB):
 n                   : 1000
 sampling_rate       : 1000.0
 data                : ['left_x', 'left_y', 'left_pupil']
 nevents             : 0
 screen_limits       : not set
 physical_screen_size: not set
 screen_eye_distance : not set
 duration_minutes    : 0.016666666666666666
 start_min           : 0.0
 end_min             : 0.01665
 parameters          : {}
 glimpse             : EyeDataDict(vars=3,n=1000,shape=(1000,)): 
  left_x (float64): 0.8876909354995632, 0.16382869242207718, 0.3047045451052535, 0.3712526256462054, -0.671312798889201...
  left_y (float64): 0.4534143924559185, -0.11460988667664473, 0.7994674729621215, -1.265084426172219, 0.777019688126354...
  left_pupil (float64): -1.008303529610942, -1.7674272682382004, -1.3504472492905297, -0.8307784244907553, -0.940278451803245...

 eyes                : ['left']
 nblinks             : {}
 blinks              : {'left': None}
 params              : {}
 History:
 *
 └ fill_time_discontinuities()

In [ ]:
d = pp.get_example_data("rlmw_002_short")
d.variables, d.eyes

(['y', 'x', 'pupil'], ['left', 'right'])

Almost all of `pypillometry`'s functions have keyword arguments `eyes=` and `variables=` that specify which eyes/variables to operate on. By default, all of the variables and eyes are processed. 

For example, we can run the `scale()` function that will re-scale the data to have mean=0 and standard devation 1. 
Here, we use the context manager `pp.loglevel("DEBUG")` to get output from `pypillometry` internals:

In [24]:
with pp.loglevel("DEBUG"):
    deye.scale(eyes="left")

pp: 12:38:57 | DEBUG    | _get_eye_var:194 | scale(): eyes=['left'], vars=['y', 'x', 'pupil']
pp: 12:38:57 | DEBUG    | scale:820 | Mean: {'left': {'y': 7.105427357601002e-18, 'x': 4.440892098500626e-19, 'pupil': -1.4210854715202004e-17}}
pp: 12:38:57 | DEBUG    | scale:821 | SD: {'left': {'y': 1.0, 'x': 1.0, 'pupil': 1.0}}


The output shows that all variables from the left eye have been processed. 



## Which functions work on which data?

Not all of `pypillometry`s functions can be applied to all variables. Functions that are specific to pupil data have the prefix `pupil_*` and functions that only work on gaze (x/y) data, have the prefix `gaze_`. The other functions will operate on all variables (which may or may not make sense, it is up to you to check!).

## Creating new variables or eyes

In some cases, new variables or "eyes" can be created. For example, we might consider to reduce a binocular dataset to one where we average the timeseries from the two eyes. In that case, we can use function `merge_eyes()`:

In [25]:
dpupil.merge_eyes(eyes=["left", "right"], variables=["pupil"], method="mean")

PupilData(mikugere, 56.3KiB):
 n               : 1000
 sampling_rate   : 1000.0
 eyes            : ['left', 'mean', 'right']
 data            : ['left_pupil', 'right_pupil', 'mean_pupil']
 nevents         : 0
 nblinks         : {}
 blinks          : {'left': None, 'mean': None, 'right': None}
 duration_minutes: 0.016666666666666666
 start_min       : 0.0
 end_min         : 0.01665
 params          : {}
 glimpse         : EyeDataDict(vars=3,n=1000,shape=(1000,)): 
  left_pupil (float64): -1.008303529610942, -1.7674272682382004, -1.3504472492905297, -0.8307784244907553, -0.940278451803245...
  right_pupil (float64): -1.0847455379130668, -0.37366426340867454, 0.008197941206316955, -0.907624569163347, 0.6937086269810662...
  mean_pupil (float64): -1.0465245337620044, -1.0705457658234374, -0.6711246540421064, -0.8692014968270512, -0.12328491241108941...

 History:
 *
 └ fill_time_discontinuities()
  └ merge_eyes(eyes=['left', 'right'],variables=['pupil'],method=mean)

We can see that a new "eye" with variable "pupil" called `mean_pupil` has been created. In this case, the original data `left_pupil` and `right_pupil` have been preserved (this can be changed by using `keep_eyes=False`).

In other cases, the package can create new variables. For example, the function `pupil_estimate_baseline()` will estimate tonic fluctuation in the pupil (see https://osf.io/preprints/psyarxiv/7ju4a_v2/) and will create a new variable `<eye>_baseline`.

In [3]:
d = pp.get_example_data("rlmw_002_short")
d.pupil_estimate_baseline()

pp: 12:55:17 | INFO     | baseline_envelope_iter_bspline:250 | Optimizing Stan model
12:55:17 - cmdstanpy - INFO - Chain [1] start processing
12:55:17 - cmdstanpy - INFO - Chain [1] done processing
12:55:17 - cmdstanpy - WARNING - The default behavior of CmdStanVB.stan_variable() will change in a future release to return the variational sample, rather than the mean.
To maintain the current behavior, pass the argument mean=True
pp: 12:55:17 | INFO     | baseline_envelope_iter_bspline:259 | Estimating PRF model (NNLS)
pp: 12:55:17 | INFO     | baseline_envelope_iter_bspline:291 | Optimizing 2nd Stan model
12:55:17 - cmdstanpy - INFO - Chain [1] start processing
12:55:17 - cmdstanpy - INFO - Chain [1] done processing
12:55:17 - cmdstanpy - WARNING - The default behavior of CmdStanVB.stan_variable() will change in a future release to return the variational sample, rather than the mean.
To maintain the current behavior, pass the argument mean=True
pp: 12:55:17 | INFO     | baseline_envelope

EyeData(test short, 2.7MiB):
 n                   : 20465
 sampling_rate       : 500.0
 data                : ['left_x', 'left_y', 'left_pupil', 'right_x', 'right_y', 'right_pupil', 'left_baseline', 'right_baseline']
 nevents             : 40
 screen_limits       : ((0, 1280), (0, 1024))
 physical_screen_size: (30, 20)
 screen_eye_distance : not set
 duration_minutes    : 0.6821666666666667
 start_min           : 0.0
 end_min             : 0.6821333333333333
 parameters          : {}
 glimpse             : EyeDataDict(vars=8,n=20465,shape=(20465,)): 
  left_x (float64): 655.6, 655.7, 655.0, 654.5, 655.0...
  left_y (float64): 599.9, 598.9, 597.6, 597.8, 597.8...
  left_pupil (float64): 1121.0, 1122.0, 1124.0, 1126.0, 1126.0...
  right_x (float64): 773.1, 773.8, 774.8, 776.3, 776.7...
  right_y (float64): 594.2, 593.9, 596.5, 597.4, 597.3...
  right_pupil (float64): 949.0, 951.0, 956.0, 959.0, 960.0...
  left_baseline (float64): 997.432053194403, 998.2207124709803, 999.0038298716057, 99